#In the beginning...there was data
Import it and clean it so we can use it

In [287]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#imports
import pandas as pd
import numpy as np
# Plotly imports
import plotly
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, plot, iplot

In [0]:
#enable plotly in colab
def enable_plotly_in_cell():
    import IPython
    from plotly.offline import init_notebook_mode
    display(IPython.core.display.HTML('''<script src="/static/components/requirejs/require.js"></script>'''))
    init_notebook_mode(connected=False)

In [290]:
#create dataframe from sequences data
sequences_df = pd.read_csv('/content/drive/My Drive/Lambda/Unit 1/Unit 1 Build/USE_2user-sequences-report-12_01_2019-02_22_2020 - USE_user-sequences-report-12_01_2019-02_22_2020.csv')
sequences_df.head()

,User Id,User Role,User Profile,User Title,User Teams,User Locked,User Custom1,User Custom2,User Custom3,User Custom4,User Custom5,User Mailbox Send Status,User Mailbox Sync Status,User Mailbox Send Last,User Mailbox Sync Last,Sequence Id,Sequence Name,Prospects Emailed,Sends,Deliveries,Opens,Clicks,Replies,Positive,Bounces,Unsubscribes,Emails/Prospect,Deliverability,Open Rate,Click Rate,Reply Rate,Positive Reply Rate,Bounce Rate,Unsubscribe Rate
0,14,SDR (US),Default,US SDR,"US Team, US SDR TEAM",False,NaN,NaN,NaN,NaN,NaN,enabled,enabled,Feb 21 2020 06:03:20 PM UTC,Feb 21 2020 09:01:20 PM UTC,30,US - SDR Sequence 2 Q419,486,1930,1874,488,0,37,0,56,0,3.86,97.10,26.04,0,1.97,0.0,2.99,0.0
1,14,SDR (US),Default,US SDR,"US Team, US SDR TEAM",False,NaN,NaN,NaN,NaN,NaN,enabled,enabled,Feb 21 2020 06:03:20 PM UTC,Feb 21 2020 09:01:20 PM UTC,41,US - Low Touch General Q419,95,309,306,101,0,11,0,3,0,3.22,99.03,33.01,0,3.59,0.0,0.98,0.0
2,14,SDR (US),Default,US SDR,"US Team, US SDR TEAM",False,NaN,NaN,NaN,NaN,NaN,enabled,enabled,Feb 21 2020 06:03:20 PM UTC,Feb 21 2020 09:01:20 PM UTC,16,US - SDR Sequence 1 Q419,455,1434,1400,422,0,22,0,34,0,3.08,97.63,30.14,0,1.57,0.0,2.43,0.0
3,14,SDR (US),Default,US SDR,"US Team, US SDR TEAM",False,NaN,NaN,NaN,NaN,NaN,enabled,enabled,Feb 21 2020 06:03:20 PM UTC,Feb 21 2020 09:01:20 PM UTC,38,US General 2 - Q419,89,191,165,29,0,2,0,26,0,1.85,86.39,17.58,0,1.21,0.0,15.76,0.0
4,14,SDR (US),Default,US SDR,"US Team, US SDR TEAM",False,NaN,NaN,NaN,NaN,NaN,enabled,enabled,Feb 21 2020 06:03:20 PM UTC,Feb 21 2020 09:01:20 PM UTC,31,US - SDR Sequence 3 Q419,132,135,132,36,0,0,0,3,0,1.00,97.78,27.27,0,0.00,NaN,2.27,0.0


In [291]:
#create a new dataframe using 'Sequence Id', 'Sequence Name', Deliveries, Opens, Replies 'Reply Rate', and 'Open Rate' columns
 sequences_2 = sequences_df[['Sequence Id','Sequence Name', 'Deliveries', 'Opens', 'Replies', 'Open Rate', 'Reply Rate']].copy()
 sequences_2.head()

,Sequence Id,Sequence Name,Deliveries,Opens,Replies,Open Rate,Reply Rate
0,30,US - SDR Sequence 2 Q419,1874,488,37,26.04,1.97
1,41,US - Low Touch General Q419,306,101,11,33.01,3.59
2,16,US - SDR Sequence 1 Q419,1400,422,22,30.14,1.57
3,38,US General 2 - Q419,165,29,2,17.58,1.21
4,31,US - SDR Sequence 3 Q419,132,36,0,27.27,0.00


In [292]:
#create a new column called 'True Reply Rate'
sequences_2['True Reply Rate'] = (sequences_2['Replies']/sequences_2['Opens'])*100
sequences_2.head()

,Sequence Id,Sequence Name,Deliveries,Opens,Replies,Open Rate,Reply Rate,True Reply Rate
0,30,US - SDR Sequence 2 Q419,1874,488,37,26.04,1.97,7.581967
1,41,US - Low Touch General Q419,306,101,11,33.01,3.59,10.891089
2,16,US - SDR Sequence 1 Q419,1400,422,22,30.14,1.57,5.213270
3,38,US General 2 - Q419,165,29,2,17.58,1.21,6.896552
4,31,US - SDR Sequence 3 Q419,132,36,0,27.27,0.00,0.000000


#Create Bar graphs of the true reply rate verse the reply rate
also find the average reply rate and the average true reply rate and assign them variables

In [392]:
sequences_2.head()

,Sequence Id,Sequence Name,Deliveries,Opens,Replies,Open Rate,Reply Rate,True Reply Rate
0,30,US - SDR Sequence 2 Q419,1874,488,37,26.04,1.97,7.581967
1,41,US - Low Touch General Q419,306,101,11,33.01,3.59,10.891089
2,16,US - SDR Sequence 1 Q419,1400,422,22,30.14,1.57,5.213270
3,38,US General 2 - Q419,165,29,2,17.58,1.21,6.896552
4,31,US - SDR Sequence 3 Q419,132,36,0,27.27,0.00,0.000000


In [393]:
#Find the average reply rate and save it as a variable
#improvement to make: format the decimals to two places
tot_del = sequences_2['Deliveries'].sum()
tot_rep = sequences_2['Replies'].sum()
avg_rep_rate = tot_rep/tot_del*100 #we multiply by one hundred to give us a percentage to work with
avg_rep_rate

2.885015655825234

In [394]:
#Find the average "true" reply rate and save it as a variable
tot_opens = sequences_2['Opens'].sum()
avg_true_rate = tot_rep/tot_opens*100
avg_true_rate

10.902872777017784

In [395]:
#store the averages in an np array
averages = np.array([avg_rep_rate, avg_true_rate])
averages

array([ 2.88501566, 10.90287278])

In [301]:
#Make the sequence names and reply rates of each sequence into arrays that can be passed to the bar chart
sequence_names = np.array(sequences_2['Sequence Name'])
my_reply_rates = np.array(sequences_2['Reply Rate'])
len(sequence_names)

61

In [0]:
# grouped bar chart
chart_sequences = sequence_names
chart_reply_rate = my_reply_rates
chart_avg = [avg_rep_rate]

mydata0 = go.Bar(x = sequence_names,
                 y = chart_reply_rate,
                 name = 'Sequence Reply Rate',
                 marker = dict(color = 'darkred'))


mydata1 = go.Bar(x = chart_sequences,
                 y = chart_avg,
                 name = 'Overall Reply Rate',
                 marker = dict(color = 'lightblue'))

In [0]:
# add some layout
mylayout = go.Layout(title = 'Sequence reply rates vs. Avg. reply rate', 
                     xaxis = dict(title = 'names'),
                     yaxis = dict(title = 'Reply rate as a percentage'),
                     barmode = 'group' )

In [304]:
enable_plotly_in_cell()
# bind the data and the layout together using go.Figure class
myfigure = go.Figure(data = [mydata0], layout = mylayout)
myfigure.show()

Output hidden; open in https://colab.research.google.com to view.

#Create Scatter plots of the reply rate verse the "True" Reply Rate

In [0]:
#Scatter plot of the reply rate
#create arrays of information to put into the scatter plot
mydata = go.Scatter(x = sequences_2['Deliveries'],
                    y = sequences_2['Replies'], 
                    mode = 'markers'
                   )

In [294]:
#create a scatter plot of number of deliveries (x) verse number of replies (y)
enable_plotly_in_cell()
myfigure = go.Figure([mydata])
myfigure

Output hidden; open in https://colab.research.google.com to view.

In [0]:
#Now create a scatter plot comparing opens and replies giving us the true reply rate
#create arrays of information to put into the scatter plot
mydata = go.Scatter(x = sequences_2['Opens'],
                    y = sequences_2['Replies'], 
                    mode = 'markers'
                    )

In [296]:
enable_plotly_in_cell()
myfigure = go.Figure([mydata])
myfigure

Output hidden; open in https://colab.research.google.com to view.

#Separate the original dataframe into smaller dataframes by sequence name

In [404]:
#Separate data by sequence and create a new summary dataframe of that sequence
#SDR Sequence 3 Q419
condition1 = sequences_2['Sequence Name'] == 'US - SDR Sequence 3 Q419'
df1 = sequences_2[condition1]
df1

,Sequence Id,Sequence Name,Deliveries,Opens,Replies,Open Rate,Reply Rate,True Reply Rate
4,31,US - SDR Sequence 3 Q419,132,36,0,27.27,0.00,0.000000
19,31,US - SDR Sequence 3 Q419,593,160,13,26.98,2.19,8.125000
34,31,US - SDR Sequence 3 Q419,561,169,22,30.12,3.92,13.017751
38,31,US - SDR Sequence 3 Q419,113,51,6,45.13,5.31,11.764706
53,31,US - SDR Sequence 3 Q419,134,52,8,38.81,5.97,15.384615


In [312]:
#Separate data by sequence
#US - SDR Email Only  - Q120                                 
condition2 = sequences_2['Sequence Name'] == 'US - SDR Email Only  - Q120'
df2 = sequences_2[condition2]
df2

,Sequence Id,Sequence Name,Deliveries,Opens,Replies,Open Rate,Reply Rate,True Reply Rate
7,134,US - SDR Email Only - Q120,4689,1247,120,26.59,2.56,9.623095
23,134,US - SDR Email Only - Q120,9190,2299,279,25.02,3.04,12.135711
35,134,US - SDR Email Only - Q120,2186,713,97,32.62,4.44,13.604488
46,134,US - SDR Email Only - Q120,149,50,12,33.56,8.05,24.000000
52,134,US - SDR Email Only - Q120,2631,747,98,28.39,3.72,13.119143


In [313]:
#Separate data by sequence
#US - SDR Sequence 1 Q419
condition3 = sequences_2['Sequence Name'] == 'US - SDR Sequence 1 Q419'
df3 = sequences_2[condition3]
df3

,Sequence Id,Sequence Name,Deliveries,Opens,Replies,Open Rate,Reply Rate,True Reply Rate
2,16,US - SDR Sequence 1 Q419,1400,422,22,30.14,1.57,5.213270
17,16,US - SDR Sequence 1 Q419,867,300,26,34.60,3.00,8.666667
32,16,US - SDR Sequence 1 Q419,750,242,23,32.27,3.07,9.504132
37,16,US - SDR Sequence 1 Q419,162,65,7,40.12,4.32,10.769231
50,16,US - SDR Sequence 1 Q419,171,60,10,35.09,5.85,16.666667


In [314]:
#Separate data by sequence
#US - SDR Sequence (Danny) Q120
condition4 = sequences_2['Sequence Name'] == 'US - SDR Sequence (Danny) Q120'
df4 = sequences_2[condition4]
df4

,Sequence Id,Sequence Name,Deliveries,Opens,Replies,Open Rate,Reply Rate,True Reply Rate
8,126,US - SDR Sequence (Danny) Q120,537,128,12,23.84,2.23,9.375000
21,126,US - SDR Sequence (Danny) Q120,176,53,4,30.11,2.27,7.547170
33,126,US - SDR Sequence (Danny) Q120,493,151,14,30.63,2.84,9.271523
40,126,US - SDR Sequence (Danny) Q120,443,116,18,26.19,4.06,15.517241
48,126,US - SDR Sequence (Danny) Q120,1555,516,76,33.18,4.89,14.728682


In [315]:
#Separate data by sequence
#US - SDR Email Only 2 Q120
condition5 = sequences_2['Sequence Name'] == 'US - SDR Email Only 2 Q120'
df5 = sequences_2[condition5]
df5

,Sequence Id,Sequence Name,Deliveries,Opens,Replies,Open Rate,Reply Rate,True Reply Rate
10,176,US - SDR Email Only 2 Q120,4229,1038,74,24.54,1.75,7.129094
26,176,US - SDR Email Only 2 Q120,4781,1115,134,23.32,2.80,12.017937
60,176,US - SDR Email Only 2 Q120,334,96,10,28.74,2.99,10.416667


In [316]:
#Separate data by sequence
#US General 1 Auto - Q419
condition6 = sequences_2['Sequence Name'] == 'US General 1 Auto - Q419'
df6 = sequences_2[condition6]
df6

,Sequence Id,Sequence Name,Deliveries,Opens,Replies,Open Rate,Reply Rate,True Reply Rate
6,53,US General 1 Auto - Q419,63,17,2,26.98,3.17,11.764706
20,53,US General 1 Auto - Q419,127,31,2,24.41,1.57,6.451613
49,53,US General 1 Auto - Q419,114,36,6,31.58,5.26,16.666667


In [317]:
#Separate data by sequence
#US - SDR - Hamid - Email Only
condition7 = sequences_2['Sequence Name'] == 'US - SDR - Hamid - Email Only'
df7 = sequences_2[condition7]
df7

,Sequence Id,Sequence Name,Deliveries,Opens,Replies,Open Rate,Reply Rate,True Reply Rate
24,161,US - SDR - Hamid - Email Only,4448,728,50,16.37,1.12,6.868132
41,161,US - SDR - Hamid - Email Only,4128,952,109,23.06,2.64,11.449580
58,161,US - SDR - Hamid - Email Only,2523,775,103,30.72,4.08,13.290323


In [318]:
#Separate data by sequence
#US SDR -  Referral Sequence - Q120
condition8 = sequences_2['Sequence Name'] == 'US SDR -  Referral Sequence - Q120'
df8 = sequences_2[condition8]
df8

,Sequence Id,Sequence Name,Deliveries,Opens,Replies,Open Rate,Reply Rate,True Reply Rate
13,114,US SDR - Referral Sequence - Q120,4,2,0,50.00,0.0,0.000000
22,114,US SDR - Referral Sequence - Q120,97,39,13,40.21,13.4,33.333333
47,114,US SDR - Referral Sequence - Q120,2,2,1,100.00,50.0,50.000000


In [319]:
#Separate data by sequence
#US - SDR - Email Only (Health/Safety Titles Only) - Q120
condition1 = sequences_2['Sequence Name'] == 'US - SDR - Email Only (Health/Safety Titles Only) - Q120'
df9 = sequences_2[condition1]
df9

,Sequence Id,Sequence Name,Deliveries,Opens,Replies,Open Rate,Reply Rate,True Reply Rate
12,213,US - SDR - Email Only (Health/Safety Titles On...,130,33,2,25.38,1.54,6.060606
28,213,US - SDR - Email Only (Health/Safety Titles On...,559,202,27,36.14,4.83,13.366337
42,213,US - SDR - Email Only (Health/Safety Titles On...,475,142,18,29.89,3.79,12.676056


In [320]:
#Separate data by sequence
#US - Low Touch General Q419
condition1 = sequences_2['Sequence Name'] == 'US - Low Touch General Q419'
df10 = sequences_2[condition1]
df10

,Sequence Id,Sequence Name,Deliveries,Opens,Replies,Open Rate,Reply Rate,True Reply Rate
1,41,US - Low Touch General Q419,306,101,11,33.01,3.59,10.891089
16,41,US - Low Touch General Q419,180,46,2,25.56,1.11,4.347826


In [321]:
#Separate data by sequence
#US - SDR - Hamid 2
condition1 = sequences_2['Sequence Name'] == 'US - SDR - Hamid 2'
df11 = sequences_2[condition1]
df11

,Sequence Id,Sequence Name,Deliveries,Opens,Replies,Open Rate,Reply Rate,True Reply Rate
44,209,US - SDR - Hamid 2,194,61,8,31.44,4.12,13.114754
59,209,US - SDR - Hamid 2,75,20,0,26.67,0.00,0.000000


In [322]:
#Separate data by sequence
#US - SDR Danny Sequence (clone 1) Q120
condition1 = sequences_2['Sequence Name'] == 'US - SDR Danny Sequence (clone 1) Q120'
df12 = sequences_2[condition1]
df12

,Sequence Id,Sequence Name,Deliveries,Opens,Replies,Open Rate,Reply Rate,True Reply Rate
36,140,US - SDR Danny Sequence (clone 1) Q120,434,128,11,29.49,2.53,8.593750
54,140,US - SDR Danny Sequence (clone 1) Q120,531,144,19,27.12,3.58,13.194444


In [323]:
#Separate data by sequence
#US - SDR - Marketing - SHRM Quiz - Hamid
condition1 = sequences_2['Sequence Name'] == 'US - SDR - Marketing - SHRM Quiz - Hamid'
df13 = sequences_2[condition1]
df13

,Sequence Id,Sequence Name,Deliveries,Opens,Replies,Open Rate,Reply Rate,True Reply Rate
9,124,US - SDR - Marketing - SHRM Quiz - Hamid,31,16,1,51.61,3.23,6.250000
39,124,US - SDR - Marketing - SHRM Quiz - Hamid,296,142,9,47.97,3.04,6.338028


In [324]:
#Separate data by sequence
#SuccessFactors Target Customers Campaign
condition1 = sequences_2['Sequence Name'] == 'SuccessFactors Target Customers Campaign'
df14 = sequences_2[condition1]
df14

,Sequence Id,Sequence Name,Deliveries,Opens,Replies,Open Rate,Reply Rate,True Reply Rate
14,195,SuccessFactors Target Customers Campaign,9,2,0,22.22,0.0,0.0
45,195,SuccessFactors Target Customers Campaign,4,2,1,50.00,25.0,50.0


In [325]:
#Separate data by sequence
#US - SDR - Marketing - CLO Content Syndication Q120
condition1 = sequences_2['Sequence Name'] == 'US - SDR - Marketing - CLO Content Syndication Q120'
df15 = sequences_2[condition1]
df15

,Sequence Id,Sequence Name,Deliveries,Opens,Replies,Open Rate,Reply Rate,True Reply Rate
15,159,US - SDR - Marketing - CLO Content Syndication...,42,13,2,30.95,4.76,15.384615
25,159,US - SDR - Marketing - CLO Content Syndication...,139,42,4,30.22,2.88,9.523810


In [326]:
#Separate data by sequence
#US General 1 -  Q419
condition1 = sequences_2['Sequence Name'] == 'US General 1 -  Q419'
df16 = sequences_2[condition1]
df16

,Sequence Id,Sequence Name,Deliveries,Opens,Replies,Open Rate,Reply Rate,True Reply Rate
5,7,US General 1 - Q419,286,83,7,29.02,2.45,8.433735
51,7,US General 1 - Q419,163,48,6,29.45,3.68,12.500000


In [327]:
#Separate data by sequence
#US - SDR Sequence 1 (clone 1)
condition1 = sequences_2['Sequence Name'] == 'US - SDR Sequence 1 (clone 1)'
df17 = sequences_2[condition1]
df17

,Sequence Id,Sequence Name,Deliveries,Opens,Replies,Open Rate,Reply Rate,True Reply Rate
11,142,US - SDR Sequence 1 (clone 1),67,22,2,32.84,2.99,9.090909
56,142,US - SDR Sequence 1 (clone 1),16,9,4,56.25,25.00,44.444444


In [328]:
#Separate data by sequence
#US - SDR Sequence 2 Q419
condition1 = sequences_2['Sequence Name'] == 'US - SDR Sequence 2 Q419'
df18 = sequences_2[condition1]
df18

,Sequence Id,Sequence Name,Deliveries,Opens,Replies,Open Rate,Reply Rate,True Reply Rate
0,30,US - SDR Sequence 2 Q419,1874,488,37,26.04,1.97,7.581967
18,30,US - SDR Sequence 2 Q419,707,164,19,23.20,2.69,11.585366


In [329]:
#Separate data by sequence
#US General 1 Auto (clone 1)
condition1 = sequences_2['Sequence Name'] == 'US General 1 Auto (clone 1)'
df19 = sequences_2[condition1]
df19

,Sequence Id,Sequence Name,Deliveries,Opens,Replies,Open Rate,Reply Rate,True Reply Rate
57,144,US General 1 Auto (clone 1),79,31,4,39.24,5.06,12.903226


In [330]:
#Separate data by sequence
#US - Asked for more info. - Q419
condition1 = sequences_2['Sequence Name'] == 'US - Asked for more info. - Q419'
df20 = sequences_2[condition1]
df20

,Sequence Id,Sequence Name,Deliveries,Opens,Replies,Open Rate,Reply Rate,True Reply Rate
31,138,US - Asked for more info. - Q419,2,2,1,100.0,50.0,50.0


In [331]:
#Separate data by sequence
#US - SDR Danny Sequence (clone 2) Q120
condition1 = sequences_2['Sequence Name'] == 'US - SDR Danny Sequence (clone 2) Q120'
df21 = sequences_2[condition1]
df21

,Sequence Id,Sequence Name,Deliveries,Opens,Replies,Open Rate,Reply Rate,True Reply Rate
55,141,US - SDR Danny Sequence (clone 2) Q120,478,155,13,32.43,2.72,8.387097


In [332]:
#Separate data by sequence
#US - SDR Sequence (Health/Safety Titles Only) - Q120
condition1 = sequences_2['Sequence Name'] == 'US - SDR Sequence (Health/Safety Titles Only) - Q120'
df22 = sequences_2[condition1]
df22

,Sequence Id,Sequence Name,Deliveries,Opens,Replies,Open Rate,Reply Rate,True Reply Rate
43,214,US - SDR Sequence (Health/Safety Titles Only) ...,78,24,3,30.77,3.85,12.5


In [333]:
#Separate data by sequence
#US - SDR Enterprise                                         
condition1 = sequences_2['Sequence Name'] == 'US - SDR Enterprise'
df23 = sequences_2[condition1]
df23

,Sequence Id,Sequence Name,Deliveries,Opens,Replies,Open Rate,Reply Rate,True Reply Rate
29,251,US - SDR Enterprise,141,60,10,42.55,7.09,16.666667


In [334]:
#Separate data by sequence
#US - SAP Successfactors Sequence                            
condition1 = sequences_2['Sequence Name'] == 'US - SAP Successfactors Sequence'
df24 = sequences_2[condition1]
df24

,Sequence Id,Sequence Name,Deliveries,Opens,Replies,Open Rate,Reply Rate,True Reply Rate
30,224,US - SAP Successfactors Sequence,2,1,0,50.0,0.0,0.0


In [335]:
#Separate data by sequence
#US General 2 - Q419
condition1 = sequences_2['Sequence Name'] == 'US General 2 - Q419'
df25 = sequences_2[condition1]
df25

,Sequence Id,Sequence Name,Deliveries,Opens,Replies,Open Rate,Reply Rate,True Reply Rate
3,38,US General 2 - Q419,165,29,2,17.58,1.21,6.896552


In [336]:
#Separate data by sequence
#US - SDR - No Show Sequence Q120
condition1 = sequences_2['Sequence Name'] == 'US - SDR - No Show Sequence Q120'
df26 = sequences_2[condition1]
df26

,Sequence Id,Sequence Name,Deliveries,Opens,Replies,Open Rate,Reply Rate,True Reply Rate
27,177,US - SDR - No Show Sequence Q120,6,2,0,33.33,0.0,0.0


#Build a new dataframe of the aggregate information of each sequence

In [0]:
#create a dataframe that holds the averages of each sequence
seq_averages = pd.DataFrame()

In [415]:
  #this code will be a function but I will test it on df1 to work out any bugs

  
  #get the total number of deliveries, opens, and replies
  deliveries = sum(df1['Deliveries'])
  opens = sum(df1['Opens'])
  replies = sum(df1['Replies'])

  #convert them to percentages
  open_rate = opens/deliveries*100
  reply_rate = replies/deliveries*100
  true_reply = replies/opens*100

  #pull out the sequence name as a single object
  sequence_name = np.array(df1['Sequence Name'])
  sequence_name = sequence_name[0]

  #add these to a new dataframe
  df1a = pd.DataFrame({'Sequence Name':[sequence_name], 'Deliveries':deliveries,
                       'Opens':opens, 'Replies':replies, 'Open Rate':open_rate,
                       'Reply Rate':reply_rate, 'True Reply Rate':true_reply}) 
  df1a

,Sequence Name,Deliveries,Opens,Replies,Open Rate,Reply Rate,True Reply Rate
0,US - SDR Sequence 3 Q419,1533,468,49,30.528376,3.196347,10.470085


In [0]:
#define a function to do all of this for the rest of the data
#This function takes the dataframe that you want to average out as the first argument
#the second argument should be a dataframe that you can concatenate by rows

def myfunc(a_df, b_df):
  deliveries = sum(a_df['Deliveries'])
  opens = sum(a_df['Opens'])
  replies = sum(a_df['Replies'])

  #convert them to percentages
  open_rate = opens/deliveries*100
  reply_rate = replies/deliveries*100
  true_reply = replies/opens*100

  #pull out the sequence name as a single object
  sequence_name = np.array(a_df['Sequence Name'])
  sequence_name = sequence_name[0]

  #add these to a new dataframe
  temp_df = pd.DataFrame({'Sequence Name':[sequence_name], 'Deliveries':deliveries,
                       'Opens':opens, 'Replies':replies, 'Open Rate':open_rate,
                       'Reply Rate':reply_rate, 'True Reply Rate':true_reply}) 
  #concatenate the temporary dataframe with b_df, an existig dataframe
  b_df = pd.concat([b_df, temp_df], axis=0)
  #combine the indices 
  #b_df = b_df.reset_index()

  return b_df



In [417]:
#create a new dataframe with the averaged values
seq_averages = myfunc(df1, seq_averages)
seq_averages = myfunc(df2, seq_averages)
seq_averages = myfunc(df3, seq_averages)
seq_averages = myfunc(df4, seq_averages)
seq_averages = myfunc(df5, seq_averages)
seq_averages = myfunc(df6, seq_averages)
seq_averages = myfunc(df7, seq_averages)
seq_averages = myfunc(df8, seq_averages)
seq_averages = myfunc(df9, seq_averages)
seq_averages = myfunc(df10, seq_averages)
seq_averages = myfunc(df11, seq_averages)
seq_averages = myfunc(df12, seq_averages)
seq_averages = myfunc(df13, seq_averages)
seq_averages = myfunc(df14, seq_averages)
seq_averages = myfunc(df15, seq_averages)
seq_averages = myfunc(df16, seq_averages)
seq_averages = myfunc(df17, seq_averages)
seq_averages = myfunc(df18, seq_averages)
seq_averages = myfunc(df19, seq_averages)
seq_averages = myfunc(df20, seq_averages)
seq_averages = myfunc(df21, seq_averages)
seq_averages = myfunc(df22, seq_averages)
seq_averages = myfunc(df23, seq_averages)
seq_averages = myfunc(df24, seq_averages)
seq_averages = myfunc(df25, seq_averages)
seq_averages = myfunc(df26, seq_averages)
seq_averages


,Sequence Name,Deliveries,Opens,Replies,Open Rate,Reply Rate,True Reply Rate
0,US - SDR Sequence 3 Q419,1533,468,49,30.528376,3.196347,10.470085
0,US - SDR Email Only - Q120,18845,5056,606,26.829398,3.215707,11.985759
0,US - SDR Sequence 1 Q419,3350,1089,88,32.507463,2.626866,8.080808
0,US - SDR Sequence (Danny) Q120,3204,964,124,30.087391,3.870162,12.863071
0,US - SDR Email Only 2 Q120,9344,2249,218,24.068921,2.333048,9.693197
0,US General 1 Auto - Q419,304,84,10,27.631579,3.289474,11.904762
0,US - SDR - Hamid - Email Only,11099,2455,262,22.119110,2.360573,10.672098
0,US SDR - Referral Sequence - Q120,103,43,14,41.747573,13.592233,32.558140
0,US - SDR - Email Only (Health/Safety Titles On...,1164,377,47,32.388316,4.037801,12.466844
0,US - Low Touch General Q419,486,147,13,30.246914,2.674897,8.843537


In [419]:
#make sure that there are 26 values
len(seq_averages)

26

#Create bar graphs using the data from seq_averages

In [421]:
seq_averages.head()

,Sequence Name,Deliveries,Opens,Replies,Open Rate,Reply Rate,True Reply Rate
0,US - SDR Sequence 3 Q419,1533,468,49,30.528376,3.196347,10.470085
0,US - SDR Email Only - Q120,18845,5056,606,26.829398,3.215707,11.985759
0,US - SDR Sequence 1 Q419,3350,1089,88,32.507463,2.626866,8.080808
0,US - SDR Sequence (Danny) Q120,3204,964,124,30.087391,3.870162,12.863071
0,US - SDR Email Only 2 Q120,9344,2249,218,24.068921,2.333048,9.693197


In [0]:
#Bar Chart



#Some random stuff....

In [396]:
#Find the average reply rate and save it as a variable
#improvement to make: format the decimals to two places
tot_del = sequences_2['Deliveries'].sum()
tot_rep = sequences_2['Replies'].sum()
avg_rep_rate = tot_rep/tot_del*100 #we multiply by one hundred to give us a percentage to work with
avg_rep_rate

2.885015655825234

In [397]:
#Find the average "true" reply rate and save it as a variable
tot_opens = sequences_2['Opens'].sum()
avg_true_rate = tot_rep/tot_opens*100
avg_true_rate

10.902872777017784

In [0]:
#Get sequence names
sequence_counts = sequences_2['Sequence Name'].value_counts()

#This is some more random stuff

In [337]:
#explore Sequence Name column
len(sequences_2['Sequence Name'].value_counts())

26

In [0]:
#create arrays of information to put into the scatter plot
mydata = go.Scatter(x = sequences_2['Sequence Name'],
                    y = sequences_2['True Reply Rate'], 
                    mode = 'markers',
                    marker = dict(size=sequences_2['Deliveries']/100))

In [339]:
#create a scatter plot of number of deliveries verse true replay rate
enable_plotly_in_cell()
myfigure = go.Figure([mydata])
myfigure

Output hidden; open in https://colab.research.google.com to view.

In [0]:
#create arrays of information to put into the scatter plot
mydata = go.Scatter(x = sequences_2['Sequence Name'],
                    y = sequences_2['Deliveries'], 
                    mode = 'markers',
                    marker = dict(size=sequences_2['True Reply Rate']))

In [341]:
enable_plotly_in_cell()
myfigure = go.Figure([mydata])
myfigure

Output hidden; open in https://colab.research.google.com to view.

In [0]:
#create a comparison between marketing cadences and other cadences with similar data


In [342]:
#create grouped bar charts comparing each cadence to the reply rate
animals=['US - SDR Email Only  - Q120',
         'US - SDR Sequence 3 Q419',
         'US - SDR Sequence 1 Q419',
         'US - SDR Sequence (Danny) Q120',
         'US - SDR - Hamid - Email Only',
         'US - SDR Email Only 2 Q120',
         'US - SDR - Email Only (Health/Safety Titles Only) - Q120',
         'US General 1 Auto - Q419',
         'US SDR -  Referral Sequence - Q120',
         'US - Low Touch General Q419',
         'US - SDR Sequence 1 (clone 1)',
         'US - SDR - Marketing - CLO Content Syndication Q120',
         'US - SDR Sequence 2 Q419',
         'US - SDR - Marketing - SHRM Quiz - Hamid',
         'SuccessFactors Target Customers Campaign',
         'US General 1 -  Q419',
         US - SDR Danny Sequence (clone 1) Q120,
         US - SDR - Hamid 2,
         US General 2 - Q419,
         US - SAP Successfactors Sequence,
         US - SDR Enterprise,
         US - SDR Sequence (Health/Safety Titles Only) - Q120,
         US - SDR Danny Sequence (clone 2) Q120,
         US - Asked for more info. - Q419,
         US General 1 Auto (clone 1),
         US - SDR - No Show Sequence Q120]
counts_a=[20, 14, 23]
counts_b=[15, 17, 8]

mydata0 = go.Bar(x = animals ,
                 y = counts_a,
                 name = 'my house',
                 marker = dict(color = 'darkred'))


mydata1 = go.Bar(x = animals,
                 y = counts_b,
                 name = 'your house',
                 marker = dict(color = 'lightblue'))

SyntaxError: ignored